In [2]:
# standard imports
import numpy as np
import pandas as pd
import random
from datetime import datetime

# import custom modules
import sys

sys.path.append("..")
import project_config

### Map Fragments to Model Scores
```bash
python inaki_map_fragments_to_model_scores.py \
    --fragment_path Data/synthesis_data 2021q3-4_Enamine_REAL_reagents_SMILES_no_salts.csv \
    --model_paths Data/property_predictors/chemprop_bbb_predictor Data/property_predictors/chemprop_drd2_predictor \
    --save_path Data/property_predictors/fragments_to_model_scores.json \
    --model_type chemprop \
    --fingerprint_type rdkit
```

## Generate Molectules Using MCTS
Chemprop with RDKit
```bash
python inaki_tree_search.py \
    --model_paths Data/property_predictors/chemprop_bbb_predictor Data/property_predictors/chemprop_drd2_predictor \
    --fragment_path data/2021q3-4_Enamine_REAL_reagents_SMILES_no_salts.csv \
    --reaction_to_reagents_path data/reaction_to_reagents_REAL_space.json \
    --fragment_to_model_score_path ckpt/AB_combined_chemprop_rdkit/fragments_to_model_scores.json \
    --save_dir generations/mcts_AB_combined_chemprop_rdkit \
    --search_type mcts \
    --model_type chemprop \
    --fingerprint_type rdkit \
    --n_rollout 20000 \
    --fragment_diversity \
    --max_reactions 1
```

Random
```bash
python tree_search.py \
    --fragment_path data/2021q3-4_Enamine_REAL_reagents_SMILES_no_salts.csv \
    --reaction_to_reagents_path data/reaction_to_reagents_REAL_space.json \
    --save_dir generations/random \
    --search_type random \
    --n_rollout 20000 \
    --max_reactions 1
```

In [ ]:
# Generate molecules with MCTS

## Make Predictions on Random Molecules

In [ ]:
# Make predictions on random molecules

## Assess Generated Molecules
1. Fragment counts
2. Maximum similarity within generated molecules
3. Maximum similarity with training data (BBB and DRD2)
4. Reaction counts (we should probably remove this or add an identifier of the reaction name to the bars)
5. Score distribution with random generation
6. Score distribution with guided generation